In [1]:
!pip install crewai crewai_tools langchain_community

In [3]:
!pip install ChromaDB

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [5]:
import os
import yaml
from crewai import Agent, Task, Crew

In [11]:
from google.colab import userdata

os.environ['SERPER_API_KEY'] = userdata.get('SERPER_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['GEMINI_API_KEY'] = userdata.get('GOOGLE_AI_STUDIO')
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')


# Loading Agents and Tasks YAML files

In [12]:
# Defining file paths for YAML configurations
files = {
    'agents': './config/agents.yaml',
    'tasks': './config/tasks.yaml'
}

# Loading configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assigning loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

# Creating Pydantic Object for Structured Output

In [13]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

# Importing CrewAI Tools

In [14]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

# Setting up Multi-LLM models

In [29]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o'
groq_llm = "groq/llama-3.3-70b-versatile"
gemini_llm = 'gemini/gemini-2.0-pro-exp-02-05'

# Creating Agents

In [38]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    # llm=gemini_llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
    # llm=gemini_llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
)

# Creating Tasks

In [39]:
# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating the Crew

In [ ]:
# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

# Kicking Off the Crew

In [41]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'The Mental Health Epidemic in Gen Z: Why ‘Quiet Quitting’ and Burnout Are Just the Tip of the Iceberg'
})

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on The Mental Health Epidemic in Gen Z: Why ‘Quiet Quitting’ and Burnout Are Just the Tip of the Iceberg. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to The Mental Health Epidemic in Gen Z: Why ‘Quiet Quitting’ and Burnout Are Just the Tip of the Iceberg.



# Agent: Lead Market Analyst
## Thought: Thought: The task at hand requires monitoring and analyzing recent news regarding the mental health epidemic in Gen Z, focusing on themes like quiet quitting and burnout and their potential market implications. To begin, I will search for the most recent news articles about this

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.57it/s]




# Agent: Chief Data Strategist
## Thought: Thought: I have gathered several relevant resources that can provide additional insights and data on the mental health epidemic, quiet quitting, and burnout in Gen Z. These resources include articles, surveys, and reports from various reputable sources. I will now extract the necessary information from these resources to form a comprehensive analysis.
1. Deloitte's Gen Z and Millennial Survey
2. ResearchGate article on quiet quitting
3. Forbes article on quiet quitting
4. Adecco's tips for supporting Gen Z workers
5. Article from Harvard Business Review
6. Article on LinkedIn regarding Gen Z employees
7. Article by CACEE on mental health and well-being in career choices
8. Jabra's thought leadership article on Gen Z job satisfaction
I'll search for detailed insights from these resources to compile the analysis.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"2023 Gen Z and Millennial Survey\", \"website\": \

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  3.14it/s]




# Agent: Chief Data Strategist
## Thought: The search on Deloitte's website did not yield detailed information about the survey within the extracted content. I will now attempt to search in another relevant resource to gather insights about Gen Z and their work-related mental health challenges.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"quiet quitting Gen Z\", \"website\": \"https://www.forbes.com\"}"
## Tool Output: 
Relevant Content:
Company Boards—Here Are All The Companies Rolling Back DEIShow MoreU.S. EditionU.S.AsiaEuropedaily cover storyHow This Legendary NBA Sixth Man Became A BillionaireByJabari Young, Forbes StaffJunior Bridgeman is now in rare air alongside Michael Jordan, Magic Johnson and LeBron James as the only NBA players with 10-figure fortunes. Crypto-Loving Robinhood Is Dogecoin Champion As It Continues To Court Young Dumb MoneyWith SEC fines and Congressional reprimands over meme stocks and gamified trading in its rearview mi

# Social Content

In [42]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

Twitter
Gen Z is facing a mental health epidemic. 😟 With
stress and burnout on the rise, 'quiet quitting'
is just the tip of the iceberg. Discover how
tailored mental health initiatives can make a
difference. [Link to blog] #GenZMentalHealth
#QuietQuitting
--------------------------------------------------
Instagram
[Image/Infographic: Visual showing percentage of
Gen Z experiencing stress and depression, designed
with vibrant graphics]   "As Gen Z marches into
the workforce, they're met with stress, burnout,
and the challenge of balancing life and work.
Explore effective strategies to support this
transformative generation! 💡 [Link in bio] #GenZ
#MentalHealthAwareness"
--------------------------------------------------
LinkedIn
Gen Z will soon dominate the workforce, but
they’re carrying the heavy load of mental health
challenges. From quiet quitting to burnout, learn
how businesses can create a supportive work
environment tailored to Gen Z. [Read the full
analysis here: Link] #GenZWo

# Blog Post

In [43]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

### Comprehensive Analysis Report on The Mental Health Epidemic in Gen Z: Quiet Quitting and Burnout

#### Introduction
The mental health epidemic within Generation Z has profound implications for both workplaces and the broader economic environment. As Gen Z soon becomes a dominant force in the labor market, businesses, investors, and policymakers must understand the underlying patterns and address the emerging risks and opportunities.

#### Key Insights

1. **Workforce Composition and Economic Influence:**
   - Gen Z is expected to represent nearly 30% of the labor force by 2025.
   - Disruptions such as global conflicts, economic instability, and the aftermath of COVID-19 significantly impact their economic participation.

2. **Mental Health Challenges:**
   - A majority of Gen Z report high stress levels (46%) and depression (35%), significantly exceeding other generations.
   - Significant factors contributing to mental strains include financial insecurity and heightened job-related pressures.

3. **Quiet Quitting Phenomenon:**
   - The trend of 'quiet quitting,' where employees do minimal work necessary to meet job requirements, is prevalent among Gen Z.
   - Factors include lack of job satisfaction, inadequate mental health support, and a pursuit of work-life balance over monetary compensation.

4. **Burnout Trends:**
   - An overwhelming 83% of frontline Gen Z workers report feeling overwhelmed at work.
   - Burnout was exacerbated by pandemic experiences and pressures, with a critical need for improved mental health strategies.

#### Market Opportunities

1. **Health and Wellness Industry:**
   - There is a growing market for mental health solutions specifically tailored to Gen Z. Companies in teletherapy, mental wellness apps, and holistic care stand to benefit significantly.
   
2. **Human Resources and Employee Benefits:**
   - Businesses that adapt to emphasize comprehensive benefits, mental health initiatives, and flexible working conditions can attract and retain Gen Z talent.
   - Innovations in HR tech for personalized employee engagement are crucial.

3. **Flexible Work and Technology Solutions:**
   - Companies offering flexible work models, including remote work and shift-trading technologies, may enjoy competitive advantages.
   - Investment in technology solutions that facilitate flexibility and enhance employee well-being is warranted.

#### Risks

1. **Economic Instability:**
   - Persistent financial insecurity among Gen Z, stemming from high living costs and student debt, may influence economic stability and consumer spending behaviors.
   
2. **Generational Workforce Disparities:**
   - Workforce polarization between generations in terms of health, benefits expectations, and work ethics could increase organizational risk.

#### Recommendations for Content Creation

1. **Educational Campaigns:**
   - Develop content that educates on recognizing and managing burnout and mental health in the workplace, highlighting effective strategies and success stories.

2. **Promotional Strategies:**
   - Position mental health resources and tools prominently within marketing materials, emphasizing their importance and effectiveness.

3. **Data-Driven Storytelling:**
   - Utilize compelling data visualizations to highlight generational disparities and trends. These visuals can foster an understanding among older generations, helping bridge the gap.

4. **Insightful Reporting:**
   - Produce deep-dive reports focusing on case studies and evolving market opportunities within mental health and wellness industries.

#### Conclusion
The alignment of business models and workplace cultures with Gen Z’s mental health needs not only addresses a critical social issue but also unlocks substantial business and market opportunities. Stakeholders must remain proactive in understanding these complex dynamics to ensure sustainability and growth.

This analysis underscores the transformative impact of Gen Z's mental health on market behaviors, urging a recalibration of strategies across various sectors.
